In [ ]:
#hide
from bs4 import BeautifulSoup
import requests
import lxml
import time
import json
import os
from os import path
import io
import core_constants as cc

# Process 247Sports Recruits & Prospects

Set the Conference you want to process as well as the folders you need

In [ ]:
#to see available conferences, uncomment out the below line and run:
#cc.get_availableConferences()
conference = 'acc'

teamDir = cc.get_htmlDir('247', conference, 'teams')
recruitDir = cc.get_htmlDir('247', conference, 'recruits')
prospectDir = cc.get_htmlDir('247', conference, 'prospects')


## Processing Engine
> This cycles through each player (recruit) and prospect profile to build our complete 247 player record

In [ ]:
all_recruits = []

for file in os.listdir(teamDir):
    gameSoup = BeautifulSoup(open(teamDir + file, "r", encoding='utf-8').read(), 'lxml')
    starter = "ri-page__list-item"
    team = file.split('_')[0]
    y = (file.split('_')[1]).split('.')[0]
    for x in gameSoup.find_all("li", class_="ri-page__list-item"):
        #append player to sqlinsert list
        player={}
        player_status = ""
        if (x.find("p", class_="commit-date") is not None):
            player_status = x.find("p", class_="commit-date").text
        if (player_status.strip() == 'Enrolled' or player_status.strip() == 'Signed'):
            player['school'] = team
            player['year'] = y
            #Name
            if (x.find("a", class_="ri-page__name-link") is not None):
                player['playerName'] = x.find("a", class_="ri-page__name-link").text
            #School/CityState
            if (x.find("span", class_="meta") is not None):
                locationInfoRaw = x.find("span", class_="meta")
                locationInfoList = locationInfoRaw.text.split("(")
                player['highSchool'] = (locationInfoList[0].strip())
                cityState = locationInfoList[1].split(', ')
                if (len(cityState) > 1):
                    #print(cityState)
                    state = cityState[1].strip()
                    #print(state.rstrip(')'))
                    player['city'] = (cityState[0].strip())
                    player['state'] = (state.rstrip(')'))
                else:
                    player['city'] = "None"
                    player['state'] = "None"
            #Position
            if (x.find("div", class_="position") is not None):
                player['position'] = ((x.find("div", class_="position").text).strip())
            #Height/Weight
            if (x.find("div", class_="metrics") is not None):
                heightWeight = x.find("div", class_="metrics").text.strip()
                height = (heightWeight.split(' / '))[0]
                inchHeightPre = height.split('-')
                if (inchHeightPre[0] != ''):
                    inchHeight = int(inchHeightPre[0])*12 + float(inchHeightPre[1])
                else:
                    inchHeight = '0.0'
                weight = (heightWeight.split(' / '))[1]
                if (weight == '-'):
                    weight = '0'
                player['height'] = (inchHeight)
                player['weight'] = (weight)
            #Getting composite rankings from the class page because of the prospect link, in some cases,
            #actually goes to the JUCO page and as a result the JUCO rankings.
            
            #Composite Rating
            if (x.find("span", class_="score") is not None):
                player['compRating'] = (x.find("span", class_="score").text)
            #Composite Stars
            ratingChildren = x.find_all("span", class_='icon-starsolid yellow')
            i = 0
            for child in ratingChildren:
                i = i + 1
            player['compStars'] = (i)
            #Composite National Rank
            if (x.find("a", class_="natrank") is not None):
                player['nationalRank'] = ((x.find("a", class_="natrank").text).strip())
            #Composite Position Rank
            if (x.find("a", class_="posrank") is not None):
                player['positionRank'] = ((x.find("a", class_="posrank").text).strip())
            #Composite State Rank
            if (x.find("a", class_="sttrank") is not None):
                player['stateRank'] = ((x.find("a", class_="sttrank").text).strip())
            
            ## We are going to get 247 rankings data from the prospect page, but nothing else
            prospectFile = prospectDir + player['playerName'] + "_" + player['school'] + "_secondhop_" + player['year'] + ".html"
            if (path.isfile(prospectFile)):
                prospectSoup = BeautifulSoup(open(prospectFile, "r", encoding='utf8').read(), 'lxml')
                ratingsSection = prospectSoup.find_all("section", class_="rankings-section")
                count = 0
                for rating in ratingsSection:
                    if (count == 1):
                        #247 Rating
                        if (rating.find("div", class_="rank-block") is not None):
                            player['247Rating'] = (rating.find("div", class_="rank-block").text)
                        #Stars
                        ratingChildren = rating.find_all("span", class_='icon-starsolid yellow')
                        i = 0
                        for child in ratingChildren:
                            i = i + 1
                        player['247Stars'] = (i)
                        #247 Rankings are stupid
                        ratingValues = rating.find_all("li", class_=None)
                        ratingColunt = 0
                        if (len(ratingValues) >= 3):
                            player['247nationalRank'] = ratingValues[0].find("strong").text
                            player['247positionRank'] = ratingValues[1].find("strong").text
                            player['247stateRank'] = ratingValues[2].find("strong").text   
                        elif (len(ratingValues) == 2):
                            if (ratingValues[0].find("strong") is not None):
                                player['247positionRank'] = ratingValues[0].find("strong").text
                            if (ratingValues[1].find("strong") is not None):
                                player['247stateRank'] = ratingValues[1].find("strong").text
                        else:
                            print("Error: " + player['playerName'])
                    count += 1
        if player:
            all_recruits.append(player)  

In [ ]:
cc.save_records('scraped_json', '247_' + conference + '.json', all_recruits)